### __Linear Regression__

In [1]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
volume_file = "./data/PIOMAS.vol.daily.1979.2020.Current.v2.1.dat"

__Load Data__

In [3]:
# year_day_to_month_day() takes a year, and the day number of the year, and from it determines and returns the month 
# and day of month.
def year_day_to_month_day(day_number, year):
    # determine if leap year needs to be taken into account
    leap_year = int(year % 4 == 0)
    # determine from day number of year which month and day of month it is
    if day_number <= 31:
        month = 1
        day = day_number
    elif day_number <= 59 + leap_year:
        month = 2
        day = day_number - 31
    elif day_number <= 90 + leap_year:
        month = 3
        day = day_number - 59 - leap_year
    elif day_number <= 120 + leap_year:
        month = 4
        day = day_number - 90 - leap_year
    elif day_number <= 151 + leap_year:
        month = 5
        day = day_number - 120 - leap_year
    elif day_number <= 181 + leap_year:
        month = 6
        day = day_number - 151 - leap_year
    elif day_number <= 212 + leap_year:
        month = 7
        day = day_number - 181 - leap_year
    elif day_number <= 243 + leap_year:
        month = 8
        day = day_number - 212 - leap_year
    elif day_number <= 273 + leap_year:
        month = 9
        day = day_number - 243 - leap_year
    elif day_number <= 304 + leap_year:
        month = 10
        day = day_number - 273 - leap_year
    elif day_number <= 334 + leap_year:
        month = 11
        day = day_number - 304 - leap_year
    else:
        month = 12
        day = day_number - 334 - leap_year
    return month, day

# load_piomas_data() will take a dataset from PIOMAS, and format it to have a column each for year, month mapped onto
# a unit circle, one for sin and one for cos, in addition to the measurement type recorded in the dataset
def load_piomas_data(file):
    # load in dataset and prepare for data manipulation
    years = []
    months_sin = []
    months_cos = []
    days = []
    measurements = []
    piomas_df = pd.read_csv(file)
    piomas_df.columns = ["Daily Data"]
    daily_data = list(piomas_df["Daily Data"])
    # for each data point, determine year, sin and cos of month mapped onto unit circle, and meausrement
    for data_point in daily_data:
        day_data = data_point.split()
        year = int(day_data[0])
        month, day = year_day_to_month_day(int(day_data[1]), year)
        month_sin = np.sin((month - 1) * (2. * np.pi/12))
        month_cos = np.cos((month - 1) * (2. * np.pi/12))
        measurement = float(day_data[2])
        years.append(year)
        months_sin.append(month_sin)
        months_cos.append(month_cos)
        days.append(day)
        measurements.append(measurement)
    data = np.transpose([years, months_sin, months_cos, measurements])
    return data

# load volume data
data = load_piomas_data(volume_file)

__Data Preparation__

In [4]:
# save feature maximums for later calculations
data_maximums = data.max(axis=0)
# normalize data for more robust linear regression fitting
data = data / data_maximums
# prepare X maxtrix with first column of ones for bias and rest of columns containing data features
X = np.ones_like(data)
X[:, 1:4] = data[:, 0:3]
# prepare y matrix with data labels
y = data[:, 3]

__Normal Equation__

In [5]:
# normalEquation() will compute and bias and feature weights using the normal equation
def normalEquation(X, y):
    # theta = inverse(X^T.X).X^T.y
    theta = np.transpose(X)
    theta = theta.dot(X)
    theta = inv(theta)
    theta = theta.dot(np.transpose(X))
    theta = theta.dot(y)
    return theta

# compute weights using normal equaion
theta_normal_equation = normalEquation(X, y)
# make predictions
y_hat = X.dot(theta_normal_equation)
# compute mean square error
MSE_normal_equation = np.mean(np.square(y - y_hat))

print("Theta computed for normal equation: ", theta_normal_equation)
print("Mean Square Error (MSE) for normal equation: ", MSE_normal_equation)

Theta computed for normal equation:  [ 18.96223064 -18.56710786   0.243309     0.0685225 ]
Mean Square Error (MSE) for normal equation:  0.0032619146605360307


In [6]:
# predict_zero_ice() will compute what year a given month will have zero volume Arctic Sea Ice, given the linear
# bias and weights within theta.  Since the data was normalized initially, the original data maximums are needed
def predict_zero_ice(theta, data_maximums):
    print("The Arctic Sea Ice will disappear in the following years for a given month:\n")
    for month in range(1, 13):
        month_sin = np.sin((month - 1) * (2. * np.pi/12)) * data_maximums[1]
        month_cos = np.cos((month - 1) * (2. * np.pi/12)) * data_maximums[2]
        year = ((-theta[0] - theta[2]*month_sin - theta[3]*month_cos) / theta[1]) * data_maximums[0]
        print("Year: {}, for Month: {}".format(int(year), month))

# make zero ice predictions with the normal equation
predict_zero_ice(theta_normal_equation, data_maximums)

The Arctic Sea Ice will disappear in the following years for a given month:

Year: 2070, for Month: 1
Year: 2082, for Month: 2
Year: 2089, for Month: 3
Year: 2089, for Month: 4
Year: 2082, for Month: 5
Year: 2069, for Month: 6
Year: 2055, for Month: 7
Year: 2043, for Month: 8
Year: 2036, for Month: 9
Year: 2036, for Month: 10
Year: 2043, for Month: 11
Year: 2056, for Month: 12


__Gradient Descent__

In [7]:
# gradientDescent() will compute and bias and feature weights using gradient descent
def gradientDescent(X, y, theta, alpha, epochs):
    # set the number of data points to n
    n = len(y)
    for i in range(epochs):
        # make predictions
        y_hat = X.dot(theta)
        # y_hat is negative, so model only works by subtracting observations from predictions, even though it should
        # actually be predictions subtracted from observations
        diff = y_hat - y
        # compute the gradients
        gradients = np.divide(np.transpose(diff).dot(X), n)
        # update the current weights
        theta = np.subtract(theta, np.multiply(alpha, gradients))
    return theta

In [8]:
# set the learning rate, number of epochs, and randomly initialize weights
learning_rate = 0.05
epochs = 5000000
theta = np.random.uniform(low=-2.0, high=2.0, size=(4,))

# compute weights using gradient descent
theta_gradient_descent = gradientDescent(X, y, theta, learning_rate, epochs)
# make predictions
y_hat = X.dot(theta_gradient_descent)
# compute mean square error
MSE_gradient_descent = np.mean(np.square(y - y_hat))

print("Theta computed for gradient descent: ", theta_gradient_descent)
print("Mean Square Error (MSE) for gradient descent: ", MSE_gradient_descent)

Theta computed for gradient descent:  [ 18.75056178 -18.35325508   0.24328079   0.06853002]
Mean Square Error (MSE) for gradient descent:  0.003263536215514401


In [9]:
# make zero ice predictions with the gradient descent
predict_zero_ice(theta_gradient_descent, data_maximums)

The Arctic Sea Ice will disappear in the following years for a given month:

Year: 2071, for Month: 1
Year: 2083, for Month: 2
Year: 2090, for Month: 3
Year: 2090, for Month: 4
Year: 2083, for Month: 5
Year: 2070, for Month: 6
Year: 2056, for Month: 7
Year: 2043, for Month: 8
Year: 2036, for Month: 9
Year: 2036, for Month: 10
Year: 2044, for Month: 11
Year: 2056, for Month: 12
